In [6]:
import pandas as pd
from datetime import datetime

In [7]:
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S')
usecols = ["id", "date", "user", "pc", "activity"]
logon_df = pd.read_csv('../data/r1/logon.csv', parse_dates=['date'], date_parser=dateparse, usecols=usecols)

In [8]:
logon_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849579 entries, 0 to 849578
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        849579 non-null  object        
 1   date      849579 non-null  datetime64[ns]
 2   user      849579 non-null  object        
 3   pc        849579 non-null  object        
 4   activity  849579 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 230.5 MB


In [ ]:
logon_df.head(10)

In [ ]:
#detect logins from dormant/new users
# use a 30-day rolling window